# In-Class Exercise: Grid Localization using Bayes Filter
<hr>

In [1]:
from robot_interface import *

# Initialize Robot to communicate with the virtual/real robot and plotter
robot = VirtualRobot()

# Initialize mapper
# Requires a VirtualRobot object as input
mapper = init_map(robot)

# Initialize your localization object
# Requires a robot VirtualRobot object and a Mapper object as inputs
loc = BaseLocalization(robot, mapper)

# Visualize the map (described using line segments) in the plotter
loc.plotter.visualize_map()

Using python version: 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0] 

Initializing Node
Initializing Virtual Robot
Initializing Mapper
 | Number of observations per grid cell:  18
 | Precaching Views...
 | Precaching Time:  12.50714111328125
Initial Pose:  (10, 10, 9)
Initializing belief with a Point mass Distribution at:  (10, 10, 9)


## 1. Find the grid indices of (0,0,0)

In [5]:
x=0
y=0 
a=0
cx, cy, cz = mapper.to_map(x,y,a)
print(cx, cy, cz)

10 10 9


<hr>

## 2. Find the the continuous world cordinates of the grid index from the previous cell

Explain the result.

In [6]:
mapper.from_map(cx, cy, cz)

(0.1, 0.1, 10.0)

<hr>

## 3. Normalize angles to the tange [-180,180]
Why do we need to do this?

In [7]:
# Angles are in degrees
for a in [30,-180,180,360,530,-10]:
    na = loc.mapper.normalize_angle(a)
    print("{:4d}   --->  {:4d}".format(a, na))

  30   --->    30
-180   --->  -180
 180   --->  -180
 360   --->     0
 530   --->   170
 -10   --->   -10


<hr>

## 4. Get Views/True Measurement Data
Get the true measurement at world coodrinates (-1,0,30)

In [8]:
cx, cy, ca = mapper.to_map(-1,0,30)

# Using the function get_views
print(loc.mapper.get_views(cx, cy, ca))
print()

# Using the numpy variable obs_views
print(loc.mapper.obs_views[cx, cy, ca, :])

[1.31769145 2.48027385 2.02193777 1.9        1.29331615 1.71129621
 1.27017059 1.11696927 1.11696927 1.27017059 1.71129621 2.23477332
 1.1        1.17059555 2.74135531 3.34863156 2.94473717 1.55379238]

[1.31769145 2.48027385 2.02193777 1.9        1.29331615 1.71129621
 1.27017059 1.11696927 1.11696927 1.27017059 1.71129621 2.23477332
 1.1        1.17059555 2.74135531 3.34863156 2.94473717 1.55379238]


<hr> 

## 5. Plotter
(For Visualization purposes, not required for the Bayes filter implementation)

5.1 Plot the true measurement data from the pose (0,0,0). <br>
5.2 Plot the observation data from the pose (0,0,0)

In [9]:
# Plots only for the pose (0,0,0)
# Add a transformation matrix to make it work for other poses
def plot_data(data, cx, cy, ca, plot_type):
    a_delta = loc.mapper.RAY_TRACING_ANGLE_INCREMENT

    rx, ry, ra = loc.mapper.from_map(cx, cy, ca)
    cur_a = ra

    for view in data:
        x = rx + view*np.cos(np.radians(cur_a))
        y = ry + view*np.sin(np.radians(cur_a))
        print("Hit Point: ", x,y)
        loc.plotter.plot_point(x,y,plot_type)

        cur_a = cur_a + a_delta
        time.sleep(1)

In [10]:
cx, cy, ca = mapper.to_map(0,0,0)

# Get the true measurements
obs_views = loc.mapper.get_views(cx, cy, ca)

# Reset Plotter and robot pose
loc.robot.reset()
loc.plotter.reset_plot()

# Plot in green
plot_data(obs_views, cx, cy, ca, GT)

 | Resetting Robot pose
Hit Point:  0.7800830152838838 0.2199169847161161
Hit Point:  0.6071796769724491 0.3928203230275509
Hit Point:  0.46500453458963054 0.5349954654103698
Hit Point:  0.31347693673800064 0.6865230632619994
Hit Point:  0.10000000000000006 0.9
Hit Point:  -0.5915434451057844 2.0
Hit Point:  -1.1845603149043786 1.630879370191243
Hit Point:  -1.4907810626334186 1.018437874733163
Hit Point:  -2.0 0.47028665948777637
Hit Point:  -2.0 -0.27028665948777675
Hit Point:  -2.0 -1.1124355652982143
Hit Point:  -0.8230095942950085 -1.0
Hit Point:  -0.30036725769282235 -1.0
Hit Point:  0.09999999999999962 -2.0
Hit Point:  0.8643374919590255 -2.0
Hit Point:  1.8621092254722875 -1.9999999999999996
Hit Point:  2.0 -0.9969655114602901
Hit Point:  2.0 -0.2350212633460835


In [10]:
cx, cy, ca = mapper.to_map(0,0,0)

# Perform rotation behaviour to get observation data at current pose
loc.get_observation_data()

# Plot in blue
plot_data(loc.obs_range_data, cx, cy, ca, ODOM)

 | Resetting Robot pose
 | Executing Observation Loop at: 30 deg/s
Hit Point:  1.102432045280151 0.27675581590966025
Hit Point:  0.8057929966557124 0.5074897766113281
Hit Point:  0.5188336271324792 0.5991464798343957
Hit Point:  0.3599364472811424 0.8141695193981953
Hit Point:  0.10000000000000006 0.9749023079872131
Hit Point:  -0.6107841765521115 2.052863475182574
Hit Point:  -1.2757065444681783 1.7395032167253177
Hit Point:  -1.3701504846944785 0.9487917780876158
Hit Point:  -1.9990482314269489 0.47011883700895696
Hit Point:  -1.7871976030181684 -0.23276385534044616
Hit Point:  -1.7461419009102126 -0.965870523452759
Hit Point:  -1.652668789402267 -1.9887493263978957
Hit Point:  -0.25729772976972154 -0.8816674445647101
Hit Point:  0.0999999999999998 -1.0647621393203734
Hit Point:  0.7951637793145302 -1.8099467864894063
Hit Point:  1.5666491883436238 -1.647884439283896
Hit Point:  2.429045737146017 -1.2446751832962046
Hit Point:  2.2069179045497624 -0.2715064727098655


## The true measurements coincide with the map obstacles.
 
## Notice how the observation data is noisy compared to the true measurements data.

<hr>

## 6. Bayes Filter

6.1 How many "for" loops in the prediction step? <br>
6.2 How many "for" loops in the update step? <br>
6.3 Do you iterate through discrete grid cells or continous coordinates? <br>
6.4 The motion and sensor model operate in the discrete grid space or the conitinous world space? <br>
6.4 Where do you use the functions "from_map()" and "to_map()"? <br>
6.5 In reference to the code, what is the belief of the robot and what is the most probable state? <br>


![title](bayes_filter.png)

<hr>

## 7. Prediction and Update Statistics and Visualizing Grid Beliefs
Check out the function **print_prediction_stats()** in class **Mapper**. <br>
Re-use the code to write your own function for debugging and/or supporting your writeup.